In [14]:
# Import all required libraries
%matplotlib inline
import category_encoders as ce
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', 50)

In [185]:
# Import data from sources
train = pd.merge(pd.read_csv('https://drive.google.com/uc?export=download&id=14ULvX0uOgftTB2s97uS8lIx1nHGQIB0P'),
                 pd.read_csv('https://drive.google.com/uc?export=download&id=1r441wLr7gKGHGLyPpKauvCuUOU556S2f'))
test = pd.read_csv('https://drive.google.com/uc?export=download&id=1wvsYl9hbRbZuIuoaLWCsW_kbcxCdocHz')
submission = pd.read_csv('https://drive.google.com/uc?export=download&id=1kfJewnmhowpUo381oSn3XqsQ6Eto23XV')
train.shape, test.shape, submission.shape

((59400, 41), (14358, 40), (14358, 2))

In [186]:
# Explore and clean data
train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.00,2011-03-14,Roman,1390,Roman,34.94,-9.86,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.00,2013-03-06,Grumeti,1399,GRUMETI,34.70,-2.15,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.00,2013-02-25,Lottery Club,686,World vision,37.46,-3.82,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.00,2013-01-28,Unicef,263,UNICEF,38.49,-11.16,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.00,2011-07-13,Action In A,0,Artisan,31.13,-1.83,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [187]:
train.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.00,59400.00,59400.00,59400.00,59400.00,59400.00,59400.00,59400.00,59400.00,59400.00
mean,37115.13,317.65,668.30,34.08,-5.71,0.47,15.30,5.63,179.91,1300.65
std,21453.13,2997.57,693.12,6.57,2.95,12.24,17.59,9.63,471.48,951.62
min,0.00,0.00,-90.00,0.00,-11.65,0.00,1.00,0.00,0.00,0.00
25%,18519.75,0.00,0.00,33.09,-8.54,0.00,5.00,2.00,0.00,0.00
50%,37061.50,0.00,369.00,34.91,-5.02,0.00,12.00,3.00,25.00,1986.00
75%,55656.50,20.00,1319.25,37.18,-3.33,0.00,17.00,5.00,215.00,2004.00
max,74247.00,350000.00,2770.00,40.35,-0.00,1776.00,99.00,80.00,30500.00,2013.00


In [188]:
train.describe(exclude='number')

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
count,59400,55765,55745,59400,59400,59029,59400,59400,59400,56066,59400,55523,31234,56344,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400
unique,356,1897,2145,37400,9,19287,21,125,2092,2,1,12,2696,2,18,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6,3
top,2011-03-15,Government Of Tanzania,DWE,none,Lake Victoria,Madukani,Iringa,Njombe,Igosi,True,GeoData Consultants Ltd,VWC,K,True,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
freq,572,9084,17402,3563,10248,508,5294,2503,307,51011,59400,36793,682,38852,26780,26780,26780,40507,52490,25348,25348,50818,50818,33186,33186,17021,17021,45794,28522,34625,32259


In [189]:
# Split train data in train and validate
train, val = train_test_split(train, train_size=0.85, test_size=0.15, 
                              stratify=train['status_group'], random_state=78)

In [190]:
# Create function to wrangle data
def wrangle(X):
    X_Clean = X.copy() # Create copy to not pass changes to main df
    
    # Convert region_code and district_code variables to str
    X_Clean['region_code'] = X_Clean['region_code'].astype('str')
    X_Clean['district_code'] = X_Clean['district_code'].astype('str')
    
    # Convert all NaN values to unknown as they are all categorical variables
    X_Clean = X_Clean.fillna('unknown')
    
    # Drop columns not needed and are duplicates
    X_Clean = X_Clean.drop(columns=['id','recorded_by','quantity_group','payment_type'])
    
    # Convert date_recorded column to show year
    dates = pd.to_datetime(X_Clean['date_recorded'])
    X_Clean['date_recorded'] = dates.dt.year
    
    # Convert 0's to NaN in construction year, gps height, logitude and latitude
    numericals = ['gps_height','longitude','construction_year', 'latitude', 'population']
    for col in numericals:
        X_Clean[col] = X_Clean[col].replace(0, np.nan)
        
    # Convert latitude almost 0 to nan
    X_Clean['latitude'] = X_Clean['latitude'].replace(X_Clean['latitude'].max(), np.nan)
    
    # Update missing numerical values using ward as basis
    for col in numericals:
        replacements = X_Clean.groupby('ward')[col].transform('mean')
        X_Clean[col] = X_Clean[col].fillna(replacements)
    
    # Now the numericals have NaN values, I will replace with means associated with region
    for col in numericals:
        replacements = X_Clean.groupby('region')[col].transform('mean')
        X_Clean[col] = X_Clean[col].fillna(replacements)
        
    # Any leftover numerical features with NaN will be updated with mean
    for col in numericals:
        replacements = X_Clean[col].mean()
        X_Clean[col] = X_Clean[col].fillna(replacements)
        
    # Create new feature age, based on date recorded minus construction date
    X_Clean['age'] = X_Clean['date_recorded'] - X_Clean['construction_year']
        
    # Return cleaned df
    return X_Clean
    

In [191]:
# Run wrangle function on train, val, and test sets
train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [ ]:
# Use PCA on numerical features to determine most 

In [192]:
print(train.describe())
print(train.describe(exclude='number'))
train_features.select_dtypes(exclude='number').nunique()

       amount_tsh  date_recorded  gps_height  longitude  latitude  \
count    50490.00       50490.00    50490.00   50490.00  50490.00   
mean      1326.24        2011.92     1016.09      35.10     -5.80   
std       4191.35           0.96      524.16       2.59      2.81   
min          0.20        2002.00      -63.00      29.61    -11.65   
25%        200.00        2011.00      633.25      33.25     -8.54   
50%        627.58        2012.00     1110.29      34.91     -5.02   
75%       1798.07        2013.00     1339.98      37.17     -3.35   
max     350000.00        2013.00     2770.00      40.35     -1.00   

       num_private  population  construction_year      age  
count     50490.00    50490.00           50490.00 50490.00  
mean         36.43      315.43            1996.37    15.55  
std          27.16      476.86              10.69    10.61  
min           1.00        1.00            1960.00    -7.00  
25%          26.48       95.00            1989.29     8.00  
50%         

funder                    1743
installer                 1960
wpt_name                 32418
basin                        9
subvillage               17792
region                      21
region_code                 27
district_code               20
lga                        125
ward                      2084
public_meeting               3
scheme_management           13
scheme_name               2586
permit                       3
extraction_type             18
extraction_type_group       13
extraction_type_class        7
management                  12
management_group             5
payment                      7
water_quality                8
quality_group                6
quantity                     5
source                      10
source_type                  7
source_class                 3
waterpoint_type              7
waterpoint_type_group        6
dtype: int64

In [193]:
# The status_group column is the target
target = 'status_group'

# Get a dataframe with all train columns except the target
train_features = train.drop(columns=[target])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 250].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features

In [194]:
# Create X matrices and Y target vectors
X_train = train[features]
Y_train = train[target]
X_val = val[features]
Y_val = val[target]
X_test = test[features]

In [195]:
# Use pipeline method to train decision tree classifier model
pipeline1 = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='mean'), 
    DecisionTreeClassifier(max_depth=20, random_state=82)
)

# Fit on train, score on val, predict on test
pipeline1.fit(X_train, Y_train)
print('Validation Accuracy', pipeline1.score(X_val, Y_val))
y_pred = pipeline1.predict(X_test)

Validation Accuracy 0.7628507295173962


In [196]:
# Use pipeline method to train random forest classifier model
pipeline2 = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    RandomForestClassifier(n_estimators=100, random_state=23, n_jobs=-1)
)

# Fit on train, score on val
pipeline2.fit(X_train, Y_train)
print('Validation Accuracy', pipeline2.score(X_val, Y_val))
y_pred2 = pipeline2.predict(X_test)

Validation Accuracy 0.8105499438832772


In [130]:
# Create submission file
sub = submission.copy()
sub['status_group'] = y_pred2
sub.to_csv('NDoshi_DS4_Sub3.csv', index = False)

In [ ]:
!head NDoshi_DS4_Sub2.csv